In [ ]:
!pip install gradio --quiet

import os
import gradio as gr
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
from PIL import Image
import torch
import numpy as np

# Updated class dictionary and colormap
class_names = [
    'urban_land', 'agriculture_land', 'rangeland', 'forest_land', 'water',
    'barren_land', 'unknown', 'building', 'land_unpaved', 'road',
    'vegetation_dubai', 'unlabeled'
]

# Color map for visualization
colormap = np.array([
    [0, 255, 255],    # Urban Land (Cyan)
    [255, 255, 0],    # Agriculture Land (Yellow)
    [255, 0, 255],    # Rangeland (Magenta)
    [0, 255, 0],      # Forest Land (Green)
    [0, 0, 255],      # Water (Blue)
    [255, 255, 255],  # Barren Land (White)
    [0, 0, 0],        # Unknown / Background (Black)
    [60, 16, 152],    # Building (Dark Purple)
    [132, 41, 246],   # Land Unpaved (Violet)
    [110, 193, 228],  # Road (Light Blue)
    [254, 221, 58],   # Vegetation Dubai (Light Yellow)
    [155, 155, 155]   # Reserved / Unused (Gray)
])

# Economic values
default_economic_values = {
    'urban_land': 1200, 'agriculture_land': 800, 'rangeland': 500,
    'forest_land': 600, 'water': 0, 'barren_land': 100, 'unknown': 0,
    'building': 1500, 'land_unpaved': 300, 'road': 1000,
    'vegetation_dubai': 400, 'unlabeled': 0
}

# Load model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b3", num_labels=len(class_names))
model.load_state_dict(torch.load("/kaggle/input/model02/pytorch/default/1/deepGlobeDubai_Segformer_mIoU54.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
model.eval()
processor = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b3")

@torch.no_grad()
def predict_segmentation(image, model, processor):
    orig_size = image.size
    inputs = processor(images=image, return_tensors="pt")
    if torch.cuda.is_available():
        model.cuda()
        inputs = {k: v.cuda() for k, v in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    seg = logits.argmax(dim=1)[0].cpu().numpy()
    return seg, orig_size, inputs['pixel_values'].shape[-2:]  # (H, W)

def decode_segmap(seg, colormap):
    h, w = seg.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)
    for idx, color in enumerate(colormap):
        rgb[seg == idx] = color
    return rgb

def flood_estimator(pre_image, post_image, pixel_area, *econ_values):
    if not isinstance(pre_image, Image.Image): pre_image = Image.open(pre_image).convert("RGB")
    if not isinstance(post_image, Image.Image): post_image = Image.open(post_image).convert("RGB")

    pre_seg, orig_size_pre, resized_size_pre = predict_segmentation(pre_image, model, processor)
    post_seg, orig_size_post, resized_size_post = predict_segmentation(post_image, model, processor)

    orig_area = orig_size_pre[0] * orig_size_pre[1]  # width * height
    resized_area = resized_size_pre[1] * resized_size_pre[0]  # width * height
    scale_factor = orig_area / resized_area
    corrected_pixel_area = pixel_area * scale_factor

    economic_inputs = {cls: val for cls, val in zip(class_names, econ_values)}

    economic_damage = {}
    lost_area = {}
    total_damage = 0

    for cls_idx, cls_name in enumerate(class_names):
        pre_area = np.sum(pre_seg == cls_idx) * corrected_pixel_area
        post_area = np.sum(post_seg == cls_idx) * corrected_pixel_area
        loss = pre_area - post_area
        if loss <= 0:
            damage = 0
        else:
            damage = loss * economic_inputs.get(cls_name, default_economic_values[cls_name])
        lost_area[cls_name] = max(loss, 0)
        economic_damage[cls_name] = damage
        total_damage += damage

    pre_seg_rgb = decode_segmap(pre_seg, colormap)
    post_seg_rgb = decode_segmap(post_seg, colormap)

    pre_img_out = Image.fromarray(pre_seg_rgb)
    post_img_out = Image.fromarray(post_seg_rgb)

    table_data = "| Class | Lost Area (sq.m) | Damage ($) |\n|---|---|---|\n"
    for cls in class_names:
        table_data += f"| {cls} | {lost_area[cls]:.2f} | ${economic_damage[cls]:.2f} |\n"
    table_data += f"\n*Total Estimated Damage: ${total_damage:,.2f}*"

    return pre_image, post_image, pre_img_out, post_img_out, table_data

# Gradio App
with gr.Blocks(theme=gr.themes.Soft()) as app:
    with gr.Row():
        gr.Image(value="/kaggle/input/logointerface/logo.jpeg", height=50, width=50,show_label=False)
        gr.Markdown("##")

    with gr.Row():
        with gr.Column(scale=1):
            pre_img = gr.Image(label="Pre-Flood Image", type="pil")
            post_img = gr.Image(label="Post-Flood Image", type="pil")
            pixel_area = gr.Number(label="Pixel Area (m²)", value=0.09)
            econ_fields = [gr.Number(label=cls, value=default_economic_values[cls]) for cls in class_names]
            submit = gr.Button("Estimate Damage")

        with gr.Column(scale=2):
            with gr.Row():
                pre_uploaded = gr.Image(label="Uploaded Pre-Flood Image")
                post_uploaded = gr.Image(label="Uploaded Post-Flood Image")
            with gr.Row():
                pre_seg_output = gr.Image(label="Pre-Flood Segmentation")
                post_seg_output = gr.Image(label="Post-Flood Segmentation")
            results_output = gr.Markdown("")

    submit.click(
        fn=flood_estimator,
        inputs=[pre_img, post_img, pixel_area] + econ_fields,
        outputs=[pre_uploaded, post_uploaded, pre_seg_output, post_seg_output, results_output]
    )

# if _name_ == '_main_':
#     app.launch()

# Launch the Gradio app inline in the Kaggle Notebook
app.launch(share=True, inline=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 26.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 89.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00


2025-05-01 04:56:00.238669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746075360.513077      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746075360.586640      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31/4213644179.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle modu

model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://613221fcd73a522d96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
